In [55]:
import urllib.request
import urllib.parse
import json
import re
import csv

In [9]:
url = "https://en.wikipedia.org/w/api.php?action=query&titles={:s}&prop=revisions&rvprop=content&format=json"

userurl = url.format("User:Almaqdisi")
userurl

'https://en.wikipedia.org/w/api.php?action=query&titles=User:Almaqdisi&prop=revisions&rvprop=content&format=json'

In [14]:
userpage =  urllib.request.urlopen(userurl).read().decode('utf-8')
print(userpage)

{"batchcomplete":"","query":{"pages":{"7036440":{"pageid":7036440,"ns":2,"title":"User:Almaqdisi","revisions":[{"contentformat":"text/x-wiki","contentmodel":"wikitext","*":"{{busy|[[User:Almaqdisi|'''Almaqdisi''']] <sup><small><font color=\"DarkBlue\">[[User_talk:Almaqdisi|talk to me]]</font></small></sup>}}\n{{Userboxtop|Professional Background|left}}\n{{user degree/PhD}}\n{{User IEEE}}\n{{Userbox\n  |border-c = #000\n  |border-s = 2\n  |id-c     = #fff\n  |id-s     = 12\n  |id-fc    = #000\n  |info-c   = #DCDCDC\n  |info-s   = 8\n  |info-fc  = #000\n  |id       = Bio\n  |info     = This user appears in ''[[Who's Who in America]]'' \u00ae\n}}\n{{Userbox\n  |border-c = #000\n  |border-s = 2\n  |id-c     = #fff\n  |id-s     = 12\n  |id-fc    = #000\n  |info-c   = #DCDCDC\n  |info-s   = 8\n  |info-fc  = #000\n  |id       = Bio\n  |info     = This user appears in ''[[Who's Who in the World]]'' \u00ae\n}}\n{{User:Scepia/published book}}\n{{User:Scepia/published academic journal}}\n{{Userbo

In [17]:
pagedict = json.loads(userpage)
pagedict.keys()

dict_keys(['query', 'batchcomplete'])

In [27]:
pagedict['query']['pages']['7036440']['revisions'][0]['*']

'{{busy|[[User:Almaqdisi|\'\'\'Almaqdisi\'\'\']] <sup><small><font color="DarkBlue">[[User_talk:Almaqdisi|talk to me]]</font></small></sup>}}\n{{Userboxtop|Professional Background|left}}\n{{user degree/PhD}}\n{{User IEEE}}\n{{Userbox\n  |border-c = #000\n  |border-s = 2\n  |id-c     = #fff\n  |id-s     = 12\n  |id-fc    = #000\n  |info-c   = #DCDCDC\n  |info-s   = 8\n  |info-fc  = #000\n  |id       = Bio\n  |info     = This user appears in \'\'[[Who\'s Who in America]]\'\' ®\n}}\n{{Userbox\n  |border-c = #000\n  |border-s = 2\n  |id-c     = #fff\n  |id-s     = 12\n  |id-fc    = #000\n  |info-c   = #DCDCDC\n  |info-s   = 8\n  |info-fc  = #000\n  |id       = Bio\n  |info     = This user appears in \'\'[[Who\'s Who in the World]]\'\' ®\n}}\n{{User:Scepia/published book}}\n{{User:Scepia/published academic journal}}\n{{Userboxbottom}}\n\n{{Babel|ar-3|en-5|fr-2|he-1|tr-3|zh-0}}\n\n{{Boxboxtop|Personal box}}\n<div class="plainlinks" style="float: left; border:solid {{{1|black}}} 1px; margin: 

In [29]:
content = pagedict['query']['pages'][list(pagedict['query']['pages'].keys())[0]]['revisions'][0]['*']

In [46]:
#p = re.compile(r'\{\{Babel\|<langinfo>\}\}')
p = re.compile(r'Babel\|(?P<langinfo>.*)\}\}')
m = p.search(content)
langinfo = m.group('langinfo')
langinfo

'ar-3|en-5|fr-2|he-1|tr-3|zh-0'

In [ ]:
user_langs = {}

In [50]:
# Build list of talk page editors
talk_csvpath = '/home/michael/school/cprose_research/wp/wikipedia/data/ipc_utf8_talkpages.csv'
talk_editors = []

with open(talk_csvpath, 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    
    # Skip first row
    next(reader)
    
    for row in reader:
        username = row[3]
        if not username in talk_editors:
            talk_editors.append(username)

print(len(talk_editors))

6604


In [56]:
editor_langinfo = {}

url = "https://en.wikipedia.org/w/api.php?action=query&titles={:s}&prop=revisions&rvprop=content&format=json"

# Extract lang info for each editor
for editor in talk_editors:
    userurl = url.format("User:{:s}".format(urllib.parse.quote(editor)))
    userpage =  urllib.request.urlopen(userurl).read().decode('utf-8')
    #pagedict = json.loads(userpage)
    #content = pagedict['query']['pages'][list(pagedict['query']['pages'].keys())[0]]['revisions'][0]['*']
    p = re.compile(r'Babel\|(?P<langinfo>.*)\}\}')
    #m = p.search(content)
    m = p.search(userpage)
    if m:
        langinfo = m.group('langinfo')
        editor_langinfo[editor] = langinfo

len(editor_langinfo)

572